<a href="https://colab.research.google.com/github/lucasnserra/CustomerSegmentation/blob/main/RFM_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###En esta Notebook abordare como obtener la Recencia,Frecuencia y el valor monetario y poder clusterizar clientes en base a los insight obtenidos.


In [2]:
# Recency: Cantidad de dias desde la ultima transaccion (Cuando el numero es mas bajo es un cliente es mas activo)
# Frecuency: Cuantas compras tiene el cliente en el periodo analizado
# Monetary Value: Cuanto gasto el cliente en el periodo analizado

In [6]:
#Importo Librerias
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#Cargo el dataset del Raw en git y determino como columna de fechas InvoiceDate para que datetime funcione
online = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/CustomerSegmentation/main/online12M.csv',parse_dates=['InvoiceDate'])
online.head()
## Las columnas principales son InvoiceDate, UnitPrice y CustomerID

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom


In [14]:
#A efectos del Dataset seleccionado vamos a estipular una fecha maxima (en un set de datos real debe ser la fecha actual) mas 1 dia para un mejor entendimiento del resultado
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days = 1)
# Creo una nueva columna que tenga el Precio por cantidad
online['TotalSum'] = online.Quantity * online.UnitPrice

In [21]:
#Calculo de la RFM
datamart = online.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'
})
# Renombro las columnas para una interpretacion mas sencilla
datamart.rename(columns={    
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'MonetaryValue'},
    inplace = True
)
datamart

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,3,25,948.70
12748,1,888,7046.16
12749,4,37,813.45
12820,4,17,268.02
12822,71,9,146.15
...,...,...,...
18280,278,2,38.70
18281,181,2,31.80
18282,8,2,30.70


In [30]:
#Creo las etiquetas RFM del 1 al 4 para segmentar los clientes por cuartiles, Teniendo en cuenta que Recency  es descendentes ya que el numero mas pequeño es mejor
r_labels = range(4,0,-1)
f_labels = range(1,5)
m_labels =range(1,5)
#Asigno esas etiquetas a los grupos de percentiles
r_groups = pd.qcut(datamart['Recency'],q=4,labels=r_labels)
f_groups = pd.qcut(datamart['Frequency'],q=4,labels=f_labels)
m_groups = pd.qcut(datamart['MonetaryValue'],q=4,labels=m_labels)
#Creo el RFM Score 
datamart = datamart.assign(R=r_groups.values,F=f_groups.values,M=m_groups.values)
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)
# Creo funcion para definir el RFM Segment 
def join_rfm(x): return str(x['R']) +str(x['F']) +str(x['M'])
datamart['RFM_Segment'] = datamart.apply(join_rfm,axis =1)
datamart.sample(5)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,RFM_Segment,RFM_Level
CustomerID,,,,,,,,,
17159,22,20,274.73,3,3,3,9,333,Middle
14776,52,27,565.32,2,4,4,10,244,Top
15909,174,1,19.80,1,1,1,3,111,Low
15561,12,4,65.46,4,1,2,7,412,Middle
14523,109,2,22.40,2,1,1,4,211,Low


In [33]:
# Creo funcion para definir Segmentar por categoria para un mayor entendimiento y tener un corte significativo
def rfm_level(df):
  if df['RFM_Score'] >= 9:
    return 'Top'
  elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9)):
      return  'Middle'
  else: 
    return 'Low'
#Le asigno la nueva clasificacion al DF
datamart ['RFM_Level'] = datamart.apply(rfm_level,axis=1)
datamart.head()


,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,RFM_Segment,RFM_Level
CustomerID,,,,,,,,,
12747,3,25,948.70,4,4,4,12,444,Top
12748,1,888,7046.16,4,4,4,12,444,Top
12749,4,37,813.45,4,4,4,12,444,Top
12820,4,17,268.02,4,3,3,10,433,Top
12822,71,9,146.15,2,2,3,7,223,Middle


In [38]:
#Creo distintos insights obtenidos de la segmentación realizada
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency':'mean',
    'MonetaryValue': ['mean','count']
}).round(1)
rfm_level_agg.sort_values(by='RFM_Level',ascending=False)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Level                                      
Top          27.0      39.4         800.8  1366
Middle       95.8       7.9         144.6  1595
Low         205.0       2.6          38.0   682